In [ ]:
!git clone https://github.com/you-ming-hu/Cardiac_Catheterization_Project
!pip install git+https://github.com/qubvel/segmentation_models.pytorch
!pip install torch==1.10.2
!pip install torchinfo
import os
os.chdir('./Cardiac_Catheterization_Project')
from IPython.display import clear_output
clear_output()
!nvidia-smi
!nvcc --version

In [ ]:
import core.utils.configuration

Config = core.utils.configuration.initialize_config()

with Config:
    Config.Data.Type = 'paired_images_with_contrast_label'
    Config.Data.Image.Preprocess = 'GrayImage' #'GrayImage' 'RGBImage' 'BGRImage' 
    Config.Data.Image.Postprocess = 'CLAHE_float' #'CLAHE_float' 'CLAHE_int'
    
    Config.Model.Task = 'SegWithClass'
    Config.Model.Param = dict(
        backbone_name = 'UniversalUnet', #'UniversalUnet' 'EffiNetB4Unet_AttDs' 'EffiNetB4Unet_AttRes'
        backbone_param = dict(
            encoder_name = 'efficientnet-b4',
            encoder_weights = 'imagenet',
            in_channels = 3,
            segmentation_dim = 4,
            classfy_dim = 8),
        seg_head_name = 'Head_1',
        seg_head_param = dict(in_channels=4),
        classify_heads_name = 'Head_1',
        classify_heads_param = dict(in_channels=8)
        )
    
    Config.Training.Settings.TrailNumber = 1
    Config.Training.Settings.Epochs = 10
    Config.Training.Settings.AmpScaleTrain = True
    
    Config.Training.Settings.Random.RootSeed = 100
    Config.Training.Settings.Random.cuDNN.Deterministic = True
    Config.Training.Settings.Random.cuDNN.Benchmark = True
    Config.Training.Dataset.Path = r'/kaggle/input/cardiac-catheterization/train'
    Config.Training.Dataset.NumWorkers = 0
    Config.Training.Dataset.PinMemory = False
    Config.Training.Dataset.PrefetchFactor = 2
    Config.Training.Dataset.Train.BatchSize = 8
    Config.Training.Dataset.Val.Ratio = 0.1
    Config.Training.Dataset.Val.BatchSize = 16
    Config.Training.Dataset.Augmentation.Prob = 'Constant(0.8)'
    Config.Training.Dataset.Augmentation.Content = [
        'GridDistortion             -p:Constant(0.8)',
        'ElasticTransform           -p:Constant(0.8)',
        'Affine                     -p:Constant(0.8)',
        'GaussNoise                 -p:Constant(0.8)',
        'Blur                       -p:Constant(0.8)',
        'Downscale                  -p:Constant(0.8)',
        'RandomBrightnessContrast   -p:Constant(0.8)'
    ]
    
    Config.Training.Losses = [
        'BCELoss2D  -subject:"mask"           -weights:"contrast_exist" -schedule:Constant(1)',
        'DiceLoss   -subject:"mask"           -weights:"contrast_exist" -schedule:Constant(1)    -smoothing:1',
        'BCELoss    -subject:"contrast_exist" -weights:None             -schedule:Constant(1)'
    ]
    
    Config.Training.Optimizer.Name = 'Adam'
    Config.Training.Optimizer.Param = {'lr':0.001}
    Config.Training.LearningRateSchedule.Name = 'Custom_1'
    Config.Training.LearningRateSchedule.Param = {'warmup_epochs':2}
    
    Config.Training.Metrics = [
        'BinaryAccuracy2D   -subject:"mask"           -weights:"contrast_exist" -threshold:0.5',
        'DiceAccuracy       -subject:"mask"           -weights:"contrast_exist" -threshold:0.5  -smoothing:1',
        'BinaryAccuracy     -subject:"contrast_exist" -weights:None             -threshold:0.5'
    ]
    
    Config.Record.Frequence = 8
    Config.Record.RootPath = 'test_save_folder'
    Config.Record.SaveModelWeights = True
    Config.Record.Note = ''

In [ ]:
%run train.py

In [ ]:
import shutil
import pathlib
import time
import torch
log_folder_path = Config.Record.RootPath
time.sleep(300)
shutil.make_archive(log_folder_path,
                    'zip',
                    log_folder_path)
print('Finish!')